# Importando as libs

In [1]:
import pandas as pd
import glob
import os
import numpy as np

# Importando os dados

In [2]:
#colunas necessárias (com base no layout) : data do pregão,código do bdi,sigla da ação,nome da ação, preço de abertura, 
#preço máximo, preço mínimo, preço de fechamento, quantidade de títulos negociados, volume de títulos negociados
nomes_colunas = ["data_pregao", "cod_bdi", "sigla_acao", "nome_acao", "preco_abertura", "preco_max",
                 "preco_min", "preco_fechamento", "qtd_negocios", "vol_negocios"
                ]
posicoes_colunas = [(2,10), (10,12), (12,24), (27,39), (56,69), (69,82), 
                    (82,95), (108,121), (152,170), (170,188)]
cwd = os.getcwd()
todos_os_arquivos =  glob.glob(cwd + "\cota_hist\COTAHIST*.zip")
df_cotacoes =pd.concat((pd.read_fwf(l,
                          compression='zip', colspecs = posicoes_colunas, names = nomes_colunas, skiprows =1, skipfooter=1)
                      for l in todos_os_arquivos), ignore_index=True)
df_cotacoes.sample(5)

,data_pregao,cod_bdi,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios
7653099,20221212,82,PETRM381,PETRE,430,469,413,413,15700,6949200
4531996,20210325,78,ITUBE266,ITUB FM,196,219,191,216,381300,76260800
164134,20150529,12,FFCI11,FII RIOB RC,153,153,152,153,24385,3717615
4291901,20210105,82,BRAPM661,BRAPE FM,175,178,130,130,43400,6973600
7568070,20220908,78,MRFGK136,MRFGE,60,60,31,31,3000,149200


In [3]:
df_cotacoes["sigla_acao"].unique()

array(['AAPL34', 'ABCB4', 'ABCB4F', ..., 'CVCBA440', 'CVCBB430',
       'CVCBB450'], dtype=object)

# Selecionando os dados - lote padrão e índice Ibovespa

In [4]:
#somente ações no lote padrão (cod_bdi=2)

df_cotacoes = df_cotacoes[df_cotacoes["cod_bdi"]==2]
df_cotacoes = df_cotacoes.drop(["cod_bdi"],1)
df_cotacoes.sample(5)

C:\Users\RAFAEL~1.PRO\AppData\Local\Temp/ipykernel_20464/2115312662.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cotacoes = df_cotacoes.drop(["cod_bdi"],1)


,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios
5993098,20220624,H1CA34,HCA HEALTHCA,47515,47672,47515,47672,37,1761195
945310,20170216,EGIE3,ENGIE BRASIL,3709,3753,3690,3715,594300,2211532300
7047328,20221125,AESB3,AES BRASIL,1008,1017,976,981,1286300,1272682300
2831423,20200513,TIET4,AES TIETE E,274,275,258,262,533500,141081900
2792209,20200316,SCHW34,SCHWAB,15840,15840,15818,15818,5350,84628500


In [5]:
#pegando somente as siglas das ações do índice Ibovespa
df_ibov_index=pd.read_csv(".\\cota_hist\\ibov.csv",sep=";")
#filtrando somente as cotações do índice Ibovespa
df_cotacoes =  pd.merge(df_cotacoes, df_ibov_index, left_on="sigla_acao", right_on="codigo").drop(["codigo","acao","qtde_teorica","part_percentual"],axis=1)
df_cotacoes.sample(5)

,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios,tipo
53,20150320,ABEV3,AMBEV S/A,1866,1890,1857,1864,16627500,31109726800,ON
52928,20201230,ENBR3,ENERGIAS BR,1985,1995,1964,1965,4501700,8865576200,ON NM
69257,20221103,ITSA4,ITAUSA,1062,1083,1046,1074,65159800,69552405900,PN EB N1
70472,20191118,ITUB4,ITAUUNIBANCO,3600,3620,3548,3552,13345400,47735593200,PN N1
22921,20191003,BRKM5,BRASKEM,3081,3111,3009,3061,1869800,5721902800,PNA N1


# Ajustando a formatação

In [6]:
#ajuste de data
df_cotacoes["data_pregao"]=pd.to_datetime(df_cotacoes["data_pregao"], format = "%Y%m%d")
df_cotacoes.sample(5)

,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios,tipo
116403,2021-08-05,VALE3,VALE,10964,10999,10790,10908,32215200,351006526600,ON NM
106041,2019-09-20,SULA11,SUL AMERICA,4650,4650,4515,4589,2228800,10191639900,UNT N2
47846,2016-06-20,ELET6,ELETROBRAS,1332,1374,1332,1366,998300,1356333700,PNB N1
28751,2019-04-29,CIEL3,CIELO,771,804,762,792,17373300,13712009400,ON EJ NM
48745,2020-02-06,ELET6,ELETROBRAS,4044,4088,3858,3926,3228600,12699907500,PNB N1


In [7]:
#ajustes preços duas casas decimais
df_cotacoes["preco_abertura"] = (df_cotacoes["preco_abertura"]/100).astype(float)
df_cotacoes["preco_max"] = (df_cotacoes["preco_max"]/100).astype(float)
df_cotacoes["preco_min"] = (df_cotacoes["preco_min"]/100).astype(float)
df_cotacoes["preco_fechamento"] = (df_cotacoes["preco_fechamento"]/100).astype(float)
df_cotacoes.sample(5)

,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios,tipo
27990,2016-03-30,CIEL3,CIELO,35.82,36.23,35.07,35.87,5683300,20322103500,ON EJ NM
28591,2018-08-29,CIEL3,CIELO,15.31,15.46,15.23,15.31,2627900,4025194100,ON EJ NM
131211,2017-04-24,AZUL4,AZUL,24.10,24.42,23.70,24.20,383100,921512200,PN N2
107510,2017-08-29,TAEE11,TAESA,22.90,22.94,22.56,22.70,711000,1615276300,UNT EDJ N2
105713,2018-05-24,SULA11,SUL AMERICA,19.09,19.53,18.82,19.10,1086500,2089098300,UNT N2


# Lidando com duplicidades e ausências

In [8]:
#removendo possíveis duplicatas
df_cotacoes = df_cotacoes.drop_duplicates()

In [9]:
#checando se existem dados vazios ou infinitos
data_to_drop = df_cotacoes.isin([np.inf, -np.inf, np.nan])
  
df_cotacoes[data_to_drop == True].count()

data_pregao         0
sigla_acao          0
nome_acao           0
preco_abertura      0
preco_max           0
preco_min           0
preco_fechamento    0
qtd_negocios        0
vol_negocios        0
tipo                0
dtype: int64

# Dataframe tratado

In [10]:
df_cotacoes.describe()

,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios
count,148259.000000,148259.000000,148259.000000,148259.000000,1.482590e+05,1.482590e+05
mean,26.728911,27.179908,26.253917,26.721135,8.133615e+06,1.655556e+10
std,20.978023,21.356586,20.620934,21.024284,1.498295e+07,3.172388e+10
min,0.670000,0.690000,0.630000,0.660000,1.000000e+02,1.400000e+04
25%,13.420000,13.670000,13.160000,13.410000,1.391650e+06,3.284075e+09
50%,22.110000,22.510000,21.700000,22.100000,3.459600e+06,7.847814e+09
75%,34.570000,35.120000,33.990000,34.560000,8.892000e+06,1.675873e+10
max,579.000000,622.000000,561.420000,621.790000,4.991948e+08,1.099640e+12


In [11]:
df_cotacoes.to_csv(cwd + "\cota_hist\cotacoes.csv.zip",index_label = "index",compression="zip")

In [12]:
df_cotacoes.sample(5)

,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,vol_negocios,tipo
76620,2020-11-05,LREN3,LOJAS RENNER,41.16,42.55,41.16,42.55,7884200,33065277400,ON NM
124818,2021-10-05,PCAR3,P.ACUCAR-CBD,23.96,25.56,23.78,25.30,4320500,10859116100,ON NM
37822,2015-12-15,CSNA3,SID NACIONAL,4.47,4.57,4.41,4.41,3039400,1359059400,ON
136683,2018-08-17,B3SA3,B3,22.79,23.09,22.50,22.55,6264700,14183479900,ON NM
110910,2015-05-28,UGPA3,ULTRAPAR,69.60,70.14,69.38,69.82,1266500,8834774600,ON NM


In [13]:
df_cotacoes["sigla_acao"].unique()

array(['ABEV3', 'ALPA4', 'ARZZ3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3',
       'BEEF3', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'BRML3', 'CCRO3',
       'CIEL3', 'CMIG4', 'CPFE3', 'CPLE6', 'CSAN3', 'CSNA3', 'CVCB3',
       'CYRE3', 'ECOR3', 'ELET3', 'ELET6', 'EMBR3', 'ENBR3', 'EQTL3',
       'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HYPE3', 'ITSA4',
       'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'MGLU3', 'MRFG3', 'MRVE3',
       'MULT3', 'PETR3', 'PETR4', 'POSI3', 'QUAL3', 'RADL3', 'RENT3',
       'SANB11', 'SBSP3', 'SLCE3', 'SMTO3', 'SULA11', 'TAEE11', 'TOTS3',
       'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'ENGI11', 'PRIO3',
       'PCAR3', 'ENEV3', 'EGIE3', 'BPAC11', 'RAIL3', 'AZUL4', 'CRFB3',
       'IRBR3', 'SUZB3', 'B3SA3', 'HAPV3', 'YDUQ3', 'COGN3', 'NTCO3',
       'LWSA3', 'SOMA3', 'PETZ3', 'TIMS3', 'CASH3', 'RRRP3', 'RDOR3',
       'CMIN3', 'ASAI3', 'AMER3', 'DXCO3', 'IGTI11', 'RAIZ4', 'VBBR3',
       'VIIA3'], dtype=object)